1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [25]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-11-30 19:59:17--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-11-30 19:59:17--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc041416d353e7342c9a5783253b.dl.dropboxusercontent.com/cd/0/inline/Bxxx4ThF4nIZn2J0H23AeEEryfhiGcDL8GNwi1AgrZt74l6hqkHgT3mfssrzWrJxaNxh1OuIwW5ILGKRNGeeyp8uh3svTznFMRHkGv8l92tv-t1I60D5vDdWfBnRyRohU9meR4SlT-TflsmjrhlbZAuzC6VbAECtBzHZ69_Cx22iDw/file# [following]
--2022-11-30 19:59:18--  https://uc041416d353e7342c9a5783253b.dl.dropboxusercontent.com/cd/0/inline/Bxxx4ThF4nIZn2J0H23AeEEryfhiGcDL8GNwi1AgrZt74l6hqkHgT3mfssrzWrJxaNxh1OuIwW5ILGKRNGeeyp8uh3svTznFM

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [1]:
import pandas as pd # standard naming convention
import numpy as np
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [3]:
x= data["BX_COUNTER"].max()
print(x)



3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [4]:
data["time"]=data["ORBIT_CNT"]* x * 25 + data["BX_COUNTER"]*25 + data["TDC_MEAS"]*25/30
print(data['time'][0:10])

0    3.446490e+14
1    3.446490e+14
2    3.446490e+14
3    3.446490e+14
4    3.446490e+14
5    3.446490e+14
6    3.446490e+14
7    3.446490e+14
8    3.446490e+14
9    3.446490e+14
Name: time, dtype: float64


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [5]:
data['FULL_TIME'] = pd.to_timedelta(data['time'])
data['FULL_TIME']

0         3 days 23:44:09.004934896
1         3 days 23:44:09.004934897
2         3 days 23:44:09.004939373
3         3 days 23:44:09.004939490
4         3 days 23:44:09.004944545
                     ...           
1310715   3 days 23:44:09.985075886
1310716   3 days 23:44:09.985075909
1310717   3 days 23:44:09.985075925
1310718   3 days 23:44:09.985076050
1310719   3 days 23:44:09.985075890
Name: FULL_TIME, Length: 1310720, dtype: timedelta64[ns]

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [10]:
data2=data.groupby(by=['TDC_CHANNEL']).sort_values(ascending=False).head(3)

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [11]:
cnt = data.ORBIT_CNT.unique().size
print(cnt)

data['ORBIT_CNT'].isna().sum()

11001


0

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

AttributeError: 'DataFrame' object has no attribute 'unique'

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [22]:
FPGA0 = pd.Series(data[data['FPGA']==0]['TDC_CHANNEL'].value_counts())
FPGA1 = pd.Series(data[data['FPGA']==1]['TDC_CHANNEL'].value_counts())

FPGA0

fpgan0= data[data["FPGA"]==0]
fpgan1= data[data["FPGA"]==1]
serie1= pd.Series(fpgan0["ORBIT_CNT"],index = fpgan0["TDC_CHANNEL"])
serie2= pd.Series(fpgan1["ORBIT_CNT"],index = fpgan1["TDC_CHANNEL"])

print(serie1)

print(serie2)


TDC_CHANNEL
123             NaN
124    3.869200e+09
63              NaN
64     3.869200e+09
64     3.869200e+09
           ...     
60              NaN
62              NaN
64     3.869200e+09
139    3.869200e+09
61              NaN
Name: ORBIT_CNT, Length: 922852, dtype: float64
TDC_CHANNEL
7     NaN
6     NaN
139   NaN
8     NaN
5     NaN
       ..
38    NaN
36    NaN
39    NaN
35    NaN
4     NaN
Name: ORBIT_CNT, Length: 387868, dtype: float64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.